In [ ]:
import pandas as pd
import altair as alt
import utils
import math

alt.data_transformers.enable('json')

aisPath = '~/fairsea_data/ais2018_chemical_tanker.csv'
chemicalIMOPath = '~/fairsea_data/imo_km.csv'

In [ ]:
# Import data
df_ais = pd.read_csv(aisPath, parse_dates=["Timestamp_datetime"])
print(f"Imported data for {df_ais.IMO.nunique()} ships")

# Filter only chemical IMOs
df_km_imo = pd.read_csv(chemicalIMOPath)
df_ais = df_ais[df_ais['IMO'].isin(df_km_imo.imo_chemical.values)]

print(f"{df_ais.IMO.nunique()} ships are chemical tankers")

In [ ]:
# Sort by time
df_ais = df_ais.sort_values(by=['Timestamp_datetime'], ascending=True)

# Remove 

In [ ]:
# Remove stationary trajectories
df_ais = df_ais.loc[df_ais['Navigational status (text)'] == 'Engine']

In [ ]:
# Group by ship
ships = [group for _name, group in df_ais.groupby("IMO")]
for df in ships:
    df['previousLatitude'] = df['Latitude'].shift(1)
    df['previousLongitude'] = df['Longitude'].shift(1)
    df['timeSinceLastPos'] = df['Timestamp_datetime'] - df['Timestamp_datetime'].shift(1)
    df['heading'] = df.apply(utils.headingFromRow, axis=1) 
    df['headingDiff'] = abs(df['heading'] - df['heading'].shift(1)) % 2* math.pi
    df['headingDiff'] = pd.concat([df['headingDiff'], 2*math.pi - df['headingDiff']], axis=1).min(axis=1)
    df.set_index('Timestamp_datetime', inplace=True)
    df['headingDiffRolling'] = df['headingDiff'].rolling(window='1H').sum()
    df.reset_index(inplace=True)

In [ ]:
brush = alt.selection_interval(encodings=['x'])

chart = alt.Chart(ships[0].drop(["timeSinceLastPos"], axis=1))

mapSource = alt.topo_feature("../data/europe.geo.json", 'geometry')
mapChart = alt.Chart(mapSource).mark_geoshape(fill=None, stroke='grey')

chart.mark_circle().encode(
    x=alt.X("Timestamp_datetime", title="Date"),
    y=alt.Y("headingDiffRolling", title="Density of turns"),
    tooltip=["Timestamp_datetime", 'IMO', 'Speed over ground']
).add_params(
    brush
).properties(
    width=1000
) & alt.layer(
    alt.Chart(mapSource).mark_geoshape(clip=True, fill='Gainsboro', stroke='grey'),
    chart.mark_line().encode(
        longitude=alt.Longitude("Longitude:Q"),
        latitude=alt.Latitude("Latitude:Q"),
        tooltip="Timestamp_datetime"
    ).transform_filter(brush)
)

Plot a heatmap of all ship routes

In [ ]:
utils.plotTrajectory(ships[0], fromDate=pd.Timestamp(2018,2,26), toDate=pd.Timestamp(2018,2,27))